In [1]:
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
from scipy import sparse
from sklearn.linear_model import LogisticRegression
import scipy.stats

In [2]:
with open('results.pkl', 'rb') as results_file:
    raw_results = pickle.load(results_file)

In [3]:
with open('tournaments.pkl', 'rb') as tournaments_file:
    raw_tournaments = pickle.load(tournaments_file)

In [4]:
with open('players.pkl', 'rb') as players_file:
    raw_players = pickle.load(players_file)

# Задание 1

Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). 

Для унификации предлагаю:

* взять в тренировочный набор турниры с dateStart из 2019 года; 

* в тестовый — турниры с dateStart из 2020 года.


Начнем с того, что из raw_results выкинем все турниры раньше 2019 года, содержащие данные о составах комманд и повопросные результаты.

In [5]:
tournament_keys = [v['id'] for k, v in raw_tournaments.items() if v['dateStart'][:4] == '2019' or v['dateStart'][:4] == '2020']

for key in tqdm(list(raw_results.keys())):
    if key not in tournament_keys or raw_results[key] == [] or not raw_results[key][0].get('mask', None) or raw_results[key][0]['mask'] == None:
        raw_results.pop(key, None)
        
len(raw_results)

100%|██████████| 5528/5528 [00:01<00:00, 3328.80it/s]


847

Удалим команды с масками ответов не равными максимальной в соревновании, удалим вопросы с пометкой `X` и заменим вопросы с пометкой `?` на `0`.

In [6]:
def check_if_mask_empty(results):
    #Проверка на наличие маски в результатах
    for tournament, teams in results.items():
        for team in teams:
            if not team['mask']:
                return True
    return False

In [7]:
def check_if_mask_small(results):
    #проверка длины маски в соревновании
    for tournament, teams in results.items():
        max_mask_lengt = 0
        for team in teams:
            if team['mask'] and len(team['mask']) > max_mask_lengt:
                max_mask_lengt = len(team['mask'])
        for team in teams:
            if team['mask'] and len(team['mask']) < max_mask_lengt:
                return True
    return False

In [8]:
def check_X_in_masks(results):
    #Проверка на наличие 'X' в результатах
    for tournament, teams in results.items():
        for team in teams:
            if team['mask'] and 'X' in team['mask']:
                return True
    return False

In [9]:
def check_qmark_in_masks(results):
    #Проверка на наличие '?' в результатах
    for tournament, teams in results.items():
        for team in teams:
            if team['mask'] and '?' in team['mask']:
                return True
    return False

In [10]:
def clean_masks(results):
    for tournament, teams in tqdm(results.items()):
        max_mask_lengt = 0
        #удалим из масок 'X' и заменим '?' на '0', а так же найдем максимальную маску в соревновании
        for team in teams:
            if team['mask'] and 'X' in team['mask']:
                team['mask'] = team['mask'].replace('X', '')
            if team['mask'] and '?' in team['mask']:
                team['mask'] = team['mask'].replace('?', '0')
            if team['mask'] and len(team['mask']) > max_mask_lengt:
                max_mask_lengt = len(team['mask'])
        #удалим команды с пустыми полями 'mask' и с короткими масками
        for idx, team in list(enumerate(teams))[::-1]:
            if not team['mask'] or len(team['mask']) < max_mask_lengt:
                teams.pop(idx)

Запустим чекеры до очистки данных:

In [11]:
print(check_if_mask_empty(raw_results))
print(check_if_mask_small(raw_results))
print(check_X_in_masks(raw_results))
print(check_qmark_in_masks(raw_results))

True
True
True
True


Запустим процесс очистки:

In [12]:
clean_masks(raw_results)

100%|██████████| 847/847 [00:00<00:00, 4375.97it/s]


Теперь чекеры не находят проблем:

In [13]:
print(check_if_mask_empty(raw_results))
print(check_if_mask_small(raw_results))
print(check_X_in_masks(raw_results))
print(check_qmark_in_masks(raw_results))

False
False
False
False


Составим обучающую и тестовую выборки по турнирам:

In [14]:
tournaments_train = []
tournaments_test = []

for tournament_id in tqdm(raw_results.keys()):
    tournament = dict()
    tournament['id'] = tournament_id
    teams_list = []
    for team in raw_results[tournament_id]:
        team_info = dict()
        team_info['id'] = team['team']['id']
        team_info['mask'] = team['mask']
        team_info['players'] = [member['player']['id'] for member in team['teamMembers']]
        teams_list.append(team_info)
    tournament['teams'] = teams_list
    
    if raw_tournaments[tournament_id]['dateStart'][:4] == '2019':
        tournaments_train.append(tournament)
    else:
        tournaments_test.append(tournament)

100%|██████████| 847/847 [00:00<00:00, 2749.20it/s]


In [15]:
len(tournaments_train), len(tournaments_test)

(674, 173)

Удалим исходные данные результатов:

In [16]:
del raw_results

# Задание 2

Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
* повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
* в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
* для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.

Присвоим каждому `id` игрока порядковый номер:

In [17]:
players = set()

for tournament in tqdm(tournaments_train):
    for team in tournament['teams']:
        for player in team['players']:
            players.add(player)

100%|██████████| 674/674 [00:00<00:00, 5849.17it/s]


In [18]:
id_to_player = {i: p for i, p in enumerate(players)}
player_to_id = {p: i for i, p in id_to_player.items()}

Составим таблицу повопросных ответов игроков размерности (все игроки + все вопросы) * (повопросные ответы).

Целевая переменная - правильность ответа (`0` или `1`)

In [19]:
%%time
player_ids = []
question_ids = []
y_labels = []
team_ids = []

questions_start_index = len(player_to_id)
for tournament in tqdm(tournaments_train):
    for team in tournament['teams']:
        team_mask = team['mask']
        for q, answer in enumerate(team_mask, questions_start_index):
            for player in team['players']:
                player_ids.append(player_to_id[player])
                question_ids.append(q)
                y_labels.append(int(answer))
                team_ids.append(team['id'])
    questions_start_index += len(team_mask)

100%|██████████| 674/674 [00:12<00:00, 52.43it/s] 

CPU times: user 12.4 s, sys: 469 ms, total: 12.9 s
Wall time: 12.9 s


In [20]:
y_train = np.array(y_labels)

In [21]:
%%time
X_train = sparse.lil_matrix((len(player_ids), questions_start_index), dtype=int)
X_train[range(len(player_ids)), player_ids] = 1
X_train[range(len(player_ids)), question_ids] = 1

CPU times: user 50.3 s, sys: 7.9 s, total: 58.2 s
Wall time: 58.1 s


In [22]:
X_train.shape, y_train.shape

((17751584, 90656), (17751584,))

Обучим на полученных данных логистическую регрессию:

In [23]:
base_model = LogisticRegression()

In [24]:
%%time
base_model.fit(X_train, y_train)

CPU times: user 4min 47s, sys: 2min 48s, total: 7min 35s
Wall time: 3min 56s


/home/dkulemin/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

В качестве рейтинга игроков возьмем первые коэффициенты модели:

In [25]:
player_rating = base_model.coef_[0][:len(player_to_id)]

Составим рейтинг лист игроков:

In [26]:
rating_table = []

for i, rating in enumerate(player_rating):
    player_id = id_to_player[i]
    rating_table.append([rating, player_id, raw_players[player_id]['name'] + ' ' + raw_players[player_id]['surname']])

In [27]:
rating_list = pd.DataFrame(sorted(rating_table, reverse=True), columns=['rating', 'player id', 'player name'])
rating_list.head(30)

,rating,player id,player name
0,3.791533,27403,Максим Руссо
1,3.635847,4270,Александра Брутер
2,3.397761,30152,Артём Сорожкин
3,3.387160,28751,Иван Семушин
4,3.320266,27822,Михаил Савченков
5,3.261366,30270,Сергей Спешков
6,3.197049,20691,Станислав Мереминский
7,3.184139,34328,Михаил Царёв
8,3.158918,37047,Мария Юнгер
9,3.158539,18036,Михаил Левандовский


# Задание 3

Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
* предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
* в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.


Оценим позицию команды в рейтинге, исходя из вероятности того, что хотябы один игрок команды правильно ответил на вопрос.

Для начала посчитаем число правильных ответов в командах:

In [28]:
tournaments_answers_count = []

for tournament in tqdm(tournaments_test):
    teams_answers_count = []
    for team in tournament['teams']:  
        test_player_ids = [player_to_id[player] for player in team['players'] if player in player_to_id.keys()]
        if len(test_player_ids):
            team_answers = list(map(int, team['mask']))
            teams_answers_count.append(sum(team_answers))
    tournaments_answers_count.append(teams_answers_count)

100%|██████████| 173/173 [00:00<00:00, 651.67it/s]


Теперь предскажем рейтинги команд, интересующие вероятности расчитаем по формуле:

$$P(team=1) = 1 - \prod P(player=0)$$

In [29]:
def tournaments_rating_predict(tournaments, to_id, start_index, model_predict_func):
    rating_pred = []
    for torunament in tqdm(tournaments):
        preds = []
        for team in torunament['teams']:
            test_player_ids = [to_id[player] for player in team['players'] if player in to_id.keys()]
            if len(test_player_ids):
                X = sparse.lil_matrix((len(test_player_ids), start_index))
                X[range(len(test_player_ids)), test_player_ids] = 1
                q = model_predict_func(X)[:, 0]
                p = 1 - q.prod()
                preds.append(p)
        rating_pred.append(preds)
    return rating_pred

In [30]:
tournaments_rating_pred = tournaments_rating_predict(
    tournaments_test, 
    player_to_id, 
    questions_start_index, 
    base_model.predict_proba
)

100%|██████████| 173/173 [00:13<00:00, 12.46it/s]


Расчитаем коеффициенты корреляции:

In [31]:
def compute_correlations(preds, true_answers):
    spearman_corrs = []
    kendall_corrs = []
    for i in range(len(true_answers)):
        if len(true_answers[i]) > 1:
            spearman_corrs.append(scipy.stats.spearmanr(true_answers[i], preds[i]).correlation)
            kendall_corrs.append(scipy.stats.kendalltau(true_answers[i], preds[i]).correlation)
    return np.mean(spearman_corrs), np.mean(kendall_corrs)

In [32]:
spearman_means, kendall_means = compute_correlations(tournaments_rating_pred, tournaments_answers_count)
print(f'Spearman mean corr: {np.round(spearman_means, 4)}')
print(f'Kendall mean corr: {np.round(kendall_means, 4)}')

Spearman mean corr: 0.7631
Kendall mean corr: 0.6084


# Задание 4

Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
* предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
* разработайте EM-схему для обучения этой модели, реализуйте её в коде;
* обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.


Вектор скрытых переменных - вероятность дать правильный ответ игроку при условии ответа команды:

$$z = P(player = 1 | team)$$

Предположим, что если команда не ответила на вопрос, тогда вероятность ответа всех игроков команды равна `0`, в получившемся векторе `z` занулим ответы у игроков таких команд

In [33]:
def sigmoid(X):
    return 1 / (1 + np.exp(-X))

In [34]:
def log_loss(y_pred, y_true):
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

In [35]:
def count_correlations():
    tournaments_rating_pred = []
    for torunament in tournaments_test:
        preds = []
        for team in torunament['teams']:
            test_player_ids = [player_to_id[player] for player in team['players'] if player in player_to_id.keys()]
            if len(test_player_ids):
                X = sparse.lil_matrix((len(test_player_ids), questions_start_index), dtype=int)
                X[range(len(test_player_ids)), test_player_ids] = 1
                X = sparse.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])
                q = 1 - sigmoid(X @ weights)
                p = 1 - q.prod()
                preds.append(p)
        tournaments_rating_pred.append(preds)

    return compute_correlations(tournaments_rating_pred, tournaments_answers_count)

Проинициируем начальные веса из baseline модели:

In [59]:
weights = np.hstack((base_model.intercept_, base_model.coef_[0]))

На E-шаге будем предсказывать вероятности ответа игроков при условии команды:

$$P(player = 1 | team = 1) = \frac{P(team = 1 | player = 1)P(player = 1)}{P(team = 1)}$$

По предположению следует, что $P(team = 1 | player = 1) = 1$, тогда:

$$P(player = 1 | team = 1) = \frac{P(player = 1)}{P(team = 1)}$$

In [37]:
def E_step(X, y, weights):
    X = sparse.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])
    preds = sigmoid(X @ weights)
    preds = np.hstack(((1 - preds).reshape(-1, 1), preds.reshape(-1, 1)))
    array = np.hstack(
        [
            np.array(team_ids).reshape(-1, 1), 
            np.array(question_ids).reshape(-1, 1), 
            preds[:, 0].reshape(-1, 1),
            preds[:, 1].reshape(-1, 1)
        ]
    )
    df_team = pd.DataFrame(array, columns=['team', 'question', 'q', 'p'])
    df_team_by_question = (
        df_team
        .groupby(['team', 'question'])
        .agg({'q': 'prod'})
        .reset_index()
    )
    df_team_by_question['p'] = 1 - df_team_by_question['q']
    df_team_by_question.drop(columns=['q'], inplace=True)
    df_team = pd.merge(df_team, df_team_by_question, on=['team', 'question'], suffixes=('_player', '_team'))
    z = (df_team['p_player'] / df_team['p_team']).clip(0, 1)
    z[y == 0] = 0
    return z

На M-шаге будем максимизировать правдоподобие, на вход будут передаваться вероятности `z`.

In [38]:
def M_step(X, y, weights, max_iters=100000, lr=20, batch_size=1000, tol=1e-04, check_loss_every=1000):
    X = sparse.hstack([np.ones(len(y)).reshape(-1, 1), X], format='csr')
    losses = []
    prev_mean_loss = np.inf

    data_len = X.shape[0]
    for i in tqdm(range(max_iters)):
        batch = np.random.choice(data_len, batch_size)
        X_batch = X[batch, :]
        y_batch = y[batch]
        preds = sigmoid(X_batch @ weights)
        losses.append(log_loss(preds, y_batch))
        if i and i % check_loss_every == 0:
            new_mean_loss = np.mean(losses)
            if (prev_mean_loss - new_mean_loss) < tol:
                print(f'Converged with mean loss: {new_mean_loss}')
                break                    
            losses = []
            prev_mean_loss = new_mean_loss

        grad = -X_batch.T @ (y_batch - preds) / len(y_batch)
        weights -= lr * grad

In [60]:
%%time
spearman_means, kendall_means = count_correlations()
print(f'Spearman mean corr: {np.round(spearman_means, 4)}')
print(f'Kendall mean corr: {np.round(kendall_means, 4)}')

best_model = weights.copy()
best_spearman = spearman_means
best_kendall = kendall_means

for _ in range(30):
    z = E_step(X_train, y_train, weights)
    M_step(X_train, z, weights)
    
    spearman_means, kendall_means = count_correlations()
    print(f'Spearman mean corr: {np.round(spearman_means, 4)}')
    print(f'Kendall mean corr: {np.round(kendall_means, 4)}')
    if spearman_means > best_spearman and kendall_means > best_kendall:
        best_model = weights.copy()
        best_spearman = spearman_means
        best_kendall = kendall_means
        print(best_spearman, best_kendall)

Spearman mean corr: 0.7631
Kendall mean corr: 0.6084


 11%|█         | 11000/100000 [06:16<50:49, 29.19it/s]


Converged with mean loss: 0.5485125062719673
Spearman mean corr: 0.7684
Kendall mean corr: 0.6132
0.7683709212278544 0.6132006701508964


 10%|█         | 10000/100000 [05:39<50:59, 29.42it/s]


Converged with mean loss: 0.48463493664496726
Spearman mean corr: 0.777
Kendall mean corr: 0.6205
0.7770151356434697 0.620535134771498


...

Converged with mean loss: 0.31351860537532433
Spearman mean corr: 0.7967
Kendall mean corr: 0.6444
0.7967072287708057 0.6443581750350759


  2%|▏         | 2000/100000 [01:09<56:31, 28.90it/s] 


Converged with mean loss: 0.31090733290131833
Spearman mean corr: 0.7965
Kendall mean corr: 0.6444


  2%|▏         | 2000/100000 [01:08<56:06, 29.11it/s] 


Converged with mean loss: 0.31791941977200777
Spearman mean corr: 0.7964
Kendall mean corr: 0.6444
CPU times: user 54min 40s, sys: 31min 35s, total: 1h 26min 16s
Wall time: 1h 25min 38s


Наблюдается постепенный рост метрик корреляции. Построим рейтинг-лист.

In [69]:
player_rating = best_model[1:len(player_to_id) + 1]
rating_table = []

for i, rating in enumerate(player_rating):
    player_id = id_to_player[i]
    rating_table.append([rating, player_id, raw_players[player_id]['name'] + ' ' + raw_players[player_id]['surname']])

rating_list = pd.DataFrame(sorted(rating_table, reverse=True), columns=['rating', 'player id', 'player name'])
rating_list.head(30)

,rating,player id,player name
0,4.284867,74001,Игорь Мокин
1,3.984195,222188,Арина Гринко
2,3.921629,34328,Михаил Царёв
3,3.902055,20691,Станислав Мереминский
4,3.883733,15727,Александр Коробейников
5,3.859326,7008,Алексей Гилёв
6,3.782159,19915,Александр Марков
7,3.760534,38196,Артём Митрофанов
8,3.730272,27403,Максим Руссо
9,3.617605,21137,Кирилл Михайлов


# Задание 5

А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).

In [71]:
question_rating = best_model[len(player_to_id) + 1:]

tournament_rating = dict()
questions_pointer = 0
for tournament in tournaments_train:
    tournament_mask = tournament['teams'][0]['mask']
    start, end = questions_pointer, questions_pointer + len(team_mask)
    tournament_rating[tournament['id']] = np.mean(question_rating[start:end])
    questions_pointer += len(tournament_mask)
    
df_tournament_rating = pd.DataFrame(
    np.array(
        sorted(
            tournament_rating.items(), 
            key=lambda x: x[1]
        )
    )[:, 0], 
    columns=['id']
)
df_tournament_rating['tournament'] = df_tournament_rating['id'].apply(lambda x: raw_tournaments[x]['name'])

In [72]:
df_tournament_rating.head(20)

,id,tournament
0,6149.0,Чемпионат Санкт-Петербурга. Первая лига
1,6147.0,Открытый зимний чемпионат ТИУ
2,6146.0,Избранное Осеннего Кубка Барнаула
3,5940.0,Чемпионат Мира. Этап 1. Группа С
4,5941.0,Чемпионат Мира. Этап 2. Группа А
5,5945.0,Чемпионат Мира. Этап 3. Группа А
6,5938.0,Чемпионат Мира. Этап 1. Группа А
7,5939.0,Чемпионат Мира. Этап 1. Группа В
8,5942.0,Чемпионат Мира. Этап 2. Группа В
9,5943.0,Чемпионат Мира. Этап 2 Группа С


In [73]:
df_tournament_rating.tail(20)

,id,tournament
654,5965.0,ОЧВР. 1 тур
655,6005.0,Кубок ректора ДВФУ
656,5010.0,Школьный Синхрон-lite. Выпуск 2.4
657,5702.0,(а)Синхрон-lite. Лига старта. Эпизод IX
658,6006.0,Зимняя кинолига
659,5112.0,Mediterranean Cup
660,6003.0,Второй тематический турнир имени Джоуи Триббиани
661,5724.0,Загадочный ларец
662,5725.0,Синхрон-lite. Выпуск XXVII
663,6001.0,Из Минска с любовью. Этап 1.


Действительно, в начале рейтинг-листа идут крупные соревнования, а в конце школьные турниры.